In [1]:
'''Importieren der notwendigen Library'''
import numpy as np

'''Klassendefinition des HammingCodes'''
class HammingCode:
    # Konstruktor für die Initialisierung des Hamming-Codes.
    def __init__(self, m):
        if (m < 2):
            raise BaseException('Eingabe ungültig, m muss grösser 2 sein!')
        else:
            self.d = 3               # Die Hammingdistanz = 3
            self.n = (2**m) - 1      # Länge des Codewortes
            self.k = (2**m) - 1 - m  # Anzahl an Datenbits
            self.p = self.n - self.k # Anzahl an Paritätsbits

    'Funktion die alle Indexe der Datenbits zurück gibt'
    def __databit_index(self):
        index_of_party_bit = [(2**i) for i in range(0, self.p)]
        return [i for i in range(1, self.n + 1) if i not in index_of_party_bit]
    
    'Funktion die alle Prüfbit-Kombinationen in einer Liste retourniert'
    def __paritybit_vectors(self):
        binaries = [format(i, 'b').zfill(self.p) for i in range(1, self.n + 1)]
        databit = self.__databit_index()
        parity_bit_list = []
        for i in range(0, self.p):
            list = []
            for j in databit:
                list.append(int(binaries[j - 1][-(i + 1)]))
            parity_bit_list.append(list)
        return parity_bit_list

    'Funktion die einen Wort in einen Vector umwandelt'
    def __word_to_vector(self, word):
        vector = np.array([], dtype=np.int64)
        for i in word:
            vector = np.append(vector, int(i))
        return vector

    'Funktion die einen Vektor in ein Wort umwandelt'
    def __vector_to_word(self, vector):
        codeword = ''
        for i in vector:
            codeword = codeword + str(i)
        return codeword
    
    'Funktion die eine n x n Einheitsmatrix zurückgibt'
    def __unity_matrix(self, n):
        Matrix = []
        for i in range(0, n):
            vector = []
            for j in range(0, n):
                if(i != j):
                    vector.append(0)
                else:
                    vector.append(1)
            Matrix.append(vector)
        return Matrix

    'Funktion um die Generatormatrix zu berechnen.'
    def get_generator_matrix(self):
        parity_bit = self.__paritybit_vectors()
        gen_matrix = self.__unity_matrix(self.k)
        for i in range(0, len(parity_bit)):
            gen_matrix.append(parity_bit[i])

        return np.array(gen_matrix)

    'Funktion um die Checkmatrix zu berechnen.'
    def get_check_matrix(self):
        parity_bit = self.__paritybit_vectors()
        unity_matrix = self.__unity_matrix(self.p)
        for i in range(0, len(parity_bit)):
            for j in range(0, len(unity_matrix[i])):
                parity_bit[i].append(unity_matrix[j][i])
        return np.array(parity_bit)

    'Funktion die ein binäres Wort codiert und das Codewort zurück liefert.'
    def encode(self, wort):
        if (len(wort) != self.k):
            raise BaseException('Ungültige Eingabe, das Wort muss eine Länge von ' + str(self.k) + ' aufweisen.')
        else:
            vectorisiert = self.__word_to_vector(wort)
            resultat = (self.get_generator_matrix() @ vectorisiert)
            # resultat = resultat % 2
            return self.__vector_to_word(resultat)

    'Funktion die das Codewort decodiert und die originale Nachricht zurückliefert.'
    def decode(self, codewort):
        codewort = np.array(self.__word_to_vector(codewort))
        checked = self.check(codewort)
        # Prüfe ob Resultat 0 Vektor ist, dann erste k-Werte zurückgeben.
        if(checked == '0' * self.p):
            return self.__vector_to_word(codewort[0:self.k])
        # Oder den Fehler korrigieren.
        else:
            checked = np.array(self.__word_to_vector(checked))
            for pos, i in enumerate(self.get_check_matrix().transpose()):
                # Fehlerindex in der Prüfmatrix ausfindig machen.
                if (np.array_equal(checked, i)):
                    # Bit an dem entsprechenden Index umwandeln (korrigieren).
                    if (codewort[pos] == 0):
                        codewort[pos] = 1 
                    else: 
                        codewort[pos] = 0
            return self.__vector_to_word(codewort[0:self.k])

    'Funktion die prüft ob ein Codewort richtig ist und den Fehler zurück gibt.'
    def check(self, codewort):
        if(len(codewort) != self.n):
            raise BaseException('Ungültige Eingabe, das Codewort muss mindestens die Länge ' + str(self.n) + 'aufweisen.')
        else:
            vectorisiert = self.__word_to_vector(codewort)
            resultat = (self.get_check_matrix() @ vectorisiert)
            resultat = resultat % 2
            return self.__vector_to_word(resultat)

In [2]:
# Instanziierung des Hammingcodes mit m = 3
hc = HammingCode(3)

# Ausgabe der  Generatormatrix
print('Generatormatrix ', hc.get_generator_matrix())

#Ausgabe der Prüfmatrix (Check-Matrix)
print('Prüfmatrix', hc.get_check_matrix())

Generatormatrix  [[1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [1 1 0 1]
 [1 0 1 1]
 [0 1 1 1]]
Prüfmatrix [[1 1 0 1 1 0 0]
 [1 0 1 1 0 1 0]
 [0 1 1 1 0 0 1]]


In [3]:
# Codierung (Encode)
print('Wort zum Codieren: ', '0101')
print('Codiertes Wort: ', hc.encode('0101'))

Wort zum Codieren:  0101
Codiertes Wort:  0101212


In [4]:
# Überprüfung / Fehlererkennung (Check) ohne Fehler
print('Codewort zum Prüfen: ', '0101010')
print('Überprüfung: ', hc.check('0101010'))
# Ergebnis 000 -> Kein Fehler im Codewort

Codewort zum Prüfen:  0101010
Überprüfung:  000


In [5]:
# Überprüfung / Fehlererkennung (Check) mit Fehler
print('Codewort zum Prüfen: ', '0101011')
print('Überprüfung: ', hc.check('0101011'))
# Ergebnis 001 -> Fehler im ersten Bit

Codewort zum Prüfen:  0101011
Überprüfung:  001


In [6]:
# Decodierung (Decode) ohne Fehlerkorrektur
print('Codewort zum Decodieren:', '0101010')
print('Encodiertes Wort: ', hc.decode('0101010'))

Codewort zum Decodieren: 0101010
Encodiertes Wort:  0101


In [7]:
# Decodierung (Decode) mit Fehlerkorrektur
print('Codewort zum Decodieren:', '0101011')
print('Encodiertes Wort: ', hc.decode('0101011'))
print('Codewort zum Decodieren:', '0101110')
print('Encodiertes Wort: ', hc.decode('0101110'))

Codewort zum Decodieren: 0101011
Encodiertes Wort:  0101
Codewort zum Decodieren: 0101110
Encodiertes Wort:  0101


In [8]:
# Decodierung (Decode) mit Fehlerkorrektur mit 2 Fehler
print('Codewort zum Decodieren:', '0101111')
print('Überprüfung: ', hc.check('0101111'))
print('Encodiertes Wort: ', hc.decode('0101111'))
print('Codiertes Wort: ', hc.encode('0001'))

Codewort zum Decodieren: 0101111
Überprüfung:  101
Encodiertes Wort:  0001
Codiertes Wort:  0001111
